In [1]:
import __init__
import numpy as np
import os
import pandas as pd
from datapath_manager import ITWDataPathManager, DataPathManager
from date_time_utils import get_date_time_from_float, convert_utc_to_local_time
from trainers import MachineLearningModelTrainer, BranchNeuralNetworkTrainer
import matplotlib.pyplot as plt
from dataloader import EmbeddingDataLoader
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import yaml, torch
from combine_features_itw import ITWFeatureCombiner

/home/nmduy/anaconda3/envs/stress/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_name = 'DCU_EXP2_ITW'
user_id = 'nvtu'
# user_id = 'nmduy'
# date = '2022-09-02'
date = '2022-09-05'
# date = '2022-09-06'

trained_dataset_name = 'DCU_NVT_EXP2'
dl_model_name = 'branch_neural_network'
ml_model_name = 'knn'
model_type = 'dependent'
window_size = 60
window_shift = 0.25

In [ ]:
dataset_path = ITWDataPathManager(dataset_name).get_dataset_path()
user_date_path = os.path.join(dataset_path, 'data', user_id, date, 'Lifelog')
stress_path = os.path.join(user_date_path, 'Stress')
relaxed_path = os.path.join(user_date_path, 'Relaxed')
low_stress_path = os.path.join(user_date_path, 'LowStress')

user_date_feature_path = os.path.join(dataset_path, 'features', user_id, date)
feature_path = os.path.join(user_date_feature_path, 'bvp_eda_temp.npy')
metadata_path = os.path.join(user_date_feature_path, 'metadata.csv')

In [ ]:
# Load features and metadata
features = np.load(feature_path)
metadata = pd.read_csv(metadata_path)

In [ ]:
def show_images(image_paths, rows, cols):
    fig, axes = plt.subplots(nrows = rows, ncols = cols, figsize = (20, 10))
    
    for i in range(rows):
        for j in range(cols):
            index = i * rows + j
            img = plt.imread(image_paths[index])
            # frame_index = os.path.basename(image_paths[index])
            # frame_index = '-'.join(image_paths[index].split('/')[-2:])
            frame_index = image_paths[index].split('/')[-2]
            axes[i, j].imshow(img)
            axes[i, j].set_title(frame_index)

In [ ]:
def get_moments_indices(moments, metadata):
    moments = [convert_utc_to_local_time(moment).timestamp() for moment in moments]
    indices = metadata.loc[metadata['date_time'].isin(moments)].index.tolist()
    return indices

In [ ]:
def get_features_from_moments(moments, features, metadata):
    indices = get_moments_indices(moments, metadata)
    return features[indices, :]

In [ ]:
stress_images = sorted([os.path.join(stress_path, image_name) for image_name in os.listdir(stress_path)])
relaxed_images = sorted([os.path.join(relaxed_path, image_name) for image_name in os.listdir(relaxed_path)])
# low_stress_images = sorted([os.path.join(relaxed_path, image_name) for image_name in os.listdir(low_stress_path)])
# # Merge relaxed with low stress
# relaxed_images.extend(low_stress_images)

In [ ]:
# show_images(stress_images[:12], 3, 4)
# X_test = features
# y_test = [0 for _ in range(X_test.shape[0] - 1)]
# y_test.append(1)
# y_test = np.array(y_test).astype(np.int64)
# test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [ ]:
stress_moments = [os.path.basename(os.path.splitext(image_name)[0]) for image_name in stress_images]
relaxed_moments = [os.path.basename(os.path.splitext(image_name)[0]) for image_name in relaxed_images]

stress_features = get_features_from_moments(stress_moments, features, metadata)
relaxed_features = get_features_from_moments(relaxed_moments, features, metadata)
X_test, y_test = np.concatenate((stress_features, relaxed_features), axis=0), np.concatenate((np.ones(len(stress_features)), np.zeros(len(relaxed_features))), axis=0)
y_test = y_test.astype(int)
test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [14]:
def get_pretrained_model(model_name):
    ds_path_manager = DataPathManager(trained_dataset_name)
    user_model_saved_path = ds_path_manager.get_saved_model_path(user_id, model_name, model_type, window_size, window_shift)
    if model_name == 'branch_neural_network':
        config_path = os.path.join(os.path.dirname(os.getcwd()), 'models', 'model_config', 'branchnn_sensor_combination.yaml')
        config_dict = yaml.safe_load(open(config_path, 'r'))
        model = BranchNeuralNetworkTrainer('.', user_model_saved_path, config_dict, target_metrics=['accuracy', 'balanced_accuracy', 'precision', 'recall'])
    else:
        model = MachineLearningModelTrainer(user_model_saved_path, model_name, eval_mode = True)
    return model

In [15]:
def evaluate(model_name, dataloader):
    y_test = dataloader.dataset.ground_truth
    model = get_pretrained_model(model_name)
    print(model.predict_and_evaluate(dataloader))
    # ConfusionMatrixDisplay.from_predictions(y_test, model.predict(dataloader), display_labels=['Relaxed', 'Stress'])

In [ ]:
evaluate(dl_model_name, test_dataloader)

In [ ]:
evaluate(ml_model_name, test_dataloader)

In [ ]:
model = get_pretrained_model(ml_model_name)
y_pred = model.predict(test_dataloader)

In [ ]:
model = get_pretrained_model(dl_model_name)
y_pred = model.predict(test_dataloader)

In [ ]:
set(y_pred)

In [ ]:
num_points = len(y_pred)
images = np.array([*stress_images, *relaxed_images])
diff_indices = [i for i in range(num_points) if y_pred[i] != y_test[i]]
diff_images = images[diff_indices]
diff_images[:5]

In [ ]:
# show_images(diff_images[:100], 10, 10)

In [ ]:
stress_indices = get_moments_indices(stress_moments, metadata)
stress_times = metadata.loc[stress_indices, 'date_time'].tolist()
stress_labels = np.ones(len(stress_indices))
relaxed_indices = get_moments_indices(relaxed_moments, metadata)
relaxed_times = metadata.loc[relaxed_indices, 'date_time'].tolist()
relaxed_labels = np.zeros(len(relaxed_indices))

In [ ]:
indices = np.array([*stress_times, *relaxed_times])
labels = np.array([*stress_labels, *relaxed_labels])
sorted_indices = np.argsort(indices)
_indices = indices[sorted_indices]
_labels = labels[sorted_indices]

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.scatter(stress_times, stress_labels, s=10, c='r', marker="x", label='Stress')
ax1.scatter(relaxed_times, relaxed_labels, s=10, c='b', marker="o", label='Relaxed')
# plt.scatter(x = _indices, y = _labels, c = ['red', 'blue'])

In [ ]:
plt.scatter(x = relaxed_indices, y = relaxed_labels, c = 'blue', label = 'Relaxed')

# --------- SESSION TEST ---------------

In [27]:
session_id = '20220906_204400'
user_id = 'nmduy'
date = '2022-09-06'
model_name = 'logistic_regression'

In [28]:
itw_feat_combiner = ITWFeatureCombiner(dataset_name)
features = itw_feat_combiner.combine_session_features(user_id, date, session_id)

In [29]:
X_test = features
y_test = np.zeros(X_test.shape[0])
test_dataloader = EmbeddingDataLoader(X_test, y_test)

In [30]:
evaluate(ml_model_name, test_dataloader)

LOAD PRETRAINED MODEL
{'accuracy': 1.0, 'balanced_accuracy': 1.0, 'precision': 0.0, 'recall': 0.0, 'f1_score': None}


/home/nmduy/anaconda3/envs/stress/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/nmduy/anaconda3/envs/stress/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
